# Tag Releases

This notebook provides the WALLABY project scientist with functionality to tag WALLABY sources for a certain release. The sources and their corresponding properties and associated data products can then be retrieved based on this release name. The process can be broken down to the following steps, for each of which we have a section in the notebook:

1. Create tag for release
2. Select sources
3. Apply tags
4. Verify and check retrival

## Admin credentials

**NOTE**: Do not add admin credentials here and run all cells. This will cause a tag to be created in the database.

In [3]:
# TODO(user): update password field here

admin_password = None
user_password = "LKaRsQrNtXZ7vN8L*6"

## Initialise Django

In [ ]:
# Import relevant libraries

import sys
import os
import django

In [ ]:
# Database access environment variables

os.environ["DJANGO_SECRET_KEY"] = "-=(gyah-@e$-ymbz02mhwu6461zv&1&8uojya413ylk!#bwa-l"
os.environ["DJANGO_SETTINGS_MODULE"] = "api.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
os.environ["DATABASE_HOST"] = "146.118.67.204"
os.environ["DATABASE_NAME"] = "wallabydb"
os.environ["DATABASE_USER"] = "wallaby_user" if admin_password is None else "admin"
os.environ["DATABASE_PASSWORD"] = user_password if admin_password is None else admin_password

In [ ]:
# Connect with SoFiAX_services Django ORM

sys.path.append('/mnt/shared/wallaby/apps/SoFiAX_services/api/')
django.setup()

In [ ]:
# Import models

from tables.models import Run, Instance, Detection, Product, Source, SourceDetection
from tables.models import Tag, TagDetection, TagSourceDetection

# 1. Create tag

Define a tag to apply to a collection of sources. This only needs a `name` and `description`. 

In [ ]:
# Create a tag called release

tag_release, _ = Tag.objects.get_or_create(
    name="Release",
    description="WALLABY release tag (or better description).",
)

# 2. Select sources

Next we will need to select the sources to tag. We will just select the first 100 sources here for some arbitrarily chosen `Run` for the example.

In [ ]:
Run.objects.all()

In [ ]:
# Example subset of sources and corresponding detections to tag as 'Release'

N = 100
run = Run.objects.get(name='NGC5044_4')
detection_subset = Detection.objects.filter(run=run)

source_detections_to_tag_release = []
for i in range(N):
    detection = detection_subset[i]
    source_name = detection.name.replace(" ", "_").replace("SoFiA", "WALLABY")
    source, _ = Source.objects.get_or_create(name=source_name)
    source_detection, _ = SourceDetection.objects.get_or_create(
        source_id=source.id,
        detection_id=detection.id
    )
    source_detections_to_tag_release.append(source_detection)

# 3. Apply tag

Apply the tag to the selected sources by using the source_detection table.

In [ ]:
# Tag SourceDetections from list

for sd in source_detections_to_tag_release:
    TagSourceDetection.objects.get_or_create(
        tag_id=tag_release.id,
        source_detection_id=sd.id,
        author="Austin Shen"
    )

In [ ]:
[Source.objects.get(id=sd.source_id) for sd in [SourceDetection.objects.get(id=tsd.source_detection_id) for tsd in TagSourceDetection.objects.filter(tag_id=Tag.objects.get(name="Release").id)]]

# 4. Retrieve

Now we have tagged bunch of the sources through this SourceDetection table. We would now like to retrieve all of these and read them into a useful table format. WALLABY users are likely to require this functionality.

In [ ]:
# Get sources with a certain tag

release_sources = [
    Source.objects.get(id=sd.source_id) for sd in [
        SourceDetection.objects.get(id=tsd.source_detection_id) for tsd in 
            TagSourceDetection.objects.filter(tag_id=Tag.objects.get(name="Release").id)
    ]
]

In [ ]:
# Verify that these sources match the tagged sources

set(release_sources) == set([Source.objects.get(id=sd.source_id) for sd in source_detections_to_tag_release])

In [ ]:
# Read as astropy table

from django.db import models
from astropy.table import Table

def astropy_table_from_tag(tag_name):
    t = Table()
    
    # get field names
    detection_field_names = [field.name for field in Detection._meta.fields if field.name != 'id' and not isinstance(field, models.ForeignKey)]
    detection_field_names.remove('name')    
    source_field_names = [field.name for field in Source._meta.fields if not isinstance(field, models.ForeignKey)]
    
    # get sources and detections
    sources = [
        Source.objects.get(id=sd.source_id) for sd in [
            SourceDetection.objects.get(id=tsd.source_detection_id) for tsd in 
                TagSourceDetection.objects.filter(tag_id=Tag.objects.get(name=tag_name).id)
        ]
    ]
    detections = [
        Detection.objects.get(id=sd.detection_id) for sd in [
            SourceDetection.objects.get(id=tsd.source_detection_id) for tsd in 
                TagSourceDetection.objects.filter(tag_id=Tag.objects.get(name=tag_name).id)
        ]
    ]
    
    # add columns to the table
    for field in detection_field_names:
        t[field] = [getattr(d, field) for d in detections]
    for field in source_field_names:
        t[field] = [getattr(s, field) for s in sources]

    return t

astropy_table_from_tag(tag_name="Release")